<span style="font-size:10pt">Copyright Jean-Luc CHARLES $-$ 2022/11 $-$ CC BY-SA 4.0 $-$ </span>
<img src="img/cc_icon_white_x2.png" width="20" style="vertical-align: middle;">
<img src="img/attribution_icon_white_x2.png" width="20" style="vertical-align: middle;">
<img src="img/sa_white_x2.png" width="20" style="vertical-align: middle;">

# Machine learning avec les modules Python tensorflow2/keras :

# Entraînement d'un réseau de neurones dense à classifier des données issues d'un banc de perçage

version 3.3 du 17 novembre 2022

<div class="alert alert-block alert-danger">
<span style="color:brown;font-family:arial;font-size:normal"> 
    Ce notebook doit être chargé dans un processus <b>jupyter notebook</b> lancé dans l'EVP <b><span style="color: rgb(80, 151, 102);">minfo_ml</span></b> créé en suivant la procédure du document <b>Consignes.pdf</b>.

## Import des modules Python requis :

In [ ]:
import os, sys

# clean tensorflow warnings:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# the seed to make random genetors repetables:
SEED = 1234 

import tensorflow as tf
from tensorflow import keras
import numpy as np

import matplotlib.pyplot as plt
print(f"Python    : {sys.version.split()[0]}")
print(f"tensorflow: {tf.__version__} incluant keras {keras.__version__}")
print(f"numpy     : {np.__version__}")

### Définition de la fonction `read_csv` qui sera utilisée pour la lecture des fichiers CSV :

In [ ]:
def read_csv(file: str, last_param_rank:int, verbose=False) -> (np.ndarray, np.ndarray, list, list):
    '''
    Lire les fichiers CSV (Comma Separated Values) en 'corrigeant les petits défauts'.
    Paramètres: 
      file:str: nom du fichier à lire
      last_param_rank:int: le rang de la dernière colonne paramètre (commençant à 0)
      verbole:bool: mode verbeux ou non-verbeux
    Renvoie:
      data:ndarray: le tableau ndarray des valeurs lues
      label:ndarray: le tableau des labels
      param:str: la liste des paramètres
      header:str: l'entête des colonnes du fichier CSV
    '''
    param, data, label, header = [], [], [], []
    with open(file, encoding='utf8') as f:
        for i, line in enumerate(f):
            if i == 0: 
                header=line.strip().split(';')
                continue
            if verbose: print(i,line)
            line = line.strip().replace(',','.').split(';')
            list_param = line[:last_param_rank+1]    # from 0 to 'last_param_rank' excluded: the parameters
            list_data  = line[last_param_rank+1:-3]  # from rank 'last_param_rank' to the end: the data
            list_label = line[-1]                    # last column: the labels
            param.append(list_param)
            data.append(list_data)
            label.append(list_label)
    data = np.array(data).astype(float)
    label = np.array(label).astype(int)
    return data, label, param, header

# 1 - Lire le fichier CSV et préparer les données labellisées

## 1.1 $-$ Lire le fichier CSV :

Ouvrir le fichier CSV `Dataset.csv` avec un tableur ; le fichier est organisé en colonnes :
- la colonne `A` (*rank: 0*) donne le numéro de l'essai de perçage,
- suivent plusieurs colonnes donnant, dans l'ordre des rangs croisants :
    - les **paramètres** de perçage, commençant avec la colonne `B` (*rank: 1*) de label `Longueur percee eprouvetteAlCFRP(mm)`, jusqu'à la colonne `O` (*rank: 14*) de label `Niveau huile`,
    - suivent les **indicateurs caractéristiques** (les *features*), commençant avec la colonne de label `KcFz`: ces indicateurs sont calculée avec les données temporelles acquises sur le banc de perçage avec les différents capteurs (capteur de force, accéléromètre, capteur de courant...).
    
Le détails des traitements permettant d'obtenir certains des indicateurs avec les données temporelles brutes fera l'objet de séances de travail dédiées.

À l'aide de la fonction `read_csv` définie ci-dessus, lire le fichier `Dataset.csv` situé dans le répertoire courant :
- en observant le fichier `Dataset.csv` ouvert dans un tableur, trouver la valeur du rang de la dernière colonne des paramètres,
- nommer `data`, `label`, `param` et `header` les objets renvoyés par la fonction...

Faire afficher l'attribut `shape` des tableaux `data` et `label`:

Est-ce que ces valeurs paraissent en cohérence avec le contenu du ficher CSV ?<br>
$\leadsto$ **Il est important de pouvoir expliquer les valeurs des dimensions des tableaux...**

Vérifier les données de la première ligne du tableau `data` par comparaison visuelle avec le tableur :

Faire afficher le contenu du tableau `label` :

Le tableau `label` montre qu'on a deux matériaux dans ce dataset :
- le matériau `0` est de l'aluminium Al7175,
- le matériau `1` est un composite CFRP (*Carbon-fiber-reinforced polymers*).<br>

## 1.1 $-$ Normaliser les données

Vous devez ici modifier chacune des lignes du tableau `data` pour les normaliser : les valeurs de chaque ligne après normalisation doivent être comprises dans l'intervalle [0; 1]... 

Vous devriez arriver à obtenir ce résultat sans écrire de boucle sur les lignes du tableau, mais en utilisant la vectorisation possible avec les tableaux `ndarray` du module *numpy*.

#### Vérification

Les valeurs de chacune des ligne du tableau `data` doivent être comprises entre 0 et 1:

In [ ]:
print(f"valeurs min des lignes du tableau 'data' normalisé : \n{data.min(axis=0)}\n" 
      f"valeurs max des lignes du tableau 'data' normalisé : \n{data.max(axis=0)}")

## 1.2 $-$ Découper les données en un jeu d'entraînement et un jeu de test

Avec l'aide de la page [sklearn.model_selection.train_test_split.html](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) utiliser la fonction `train_test_split` pour découper le tableau `data` en deux jeux de données labellisées :
- `data_train` et `label_train` $\leadsto$ données et labels d'entraînement,
- `data_val` et `label_val`  $\leadsto$  donnnées et labels de validation.<br>
On pourra par exemple regrouper 20% des données et labels pour le jeu de validation.

Il est important de passer à `train_test_split` les arguments :
- `stratify=label`, afin de répartir équitablement toutes les classes sur les deux jeux de données,
- `shuffle=True`, pour mélanger les données,
- `random_state=SEED`, pour obtenir un mélange aléatoire des données qui soit reproductible...


Vérification des dimensions des tableaux:

In [ ]:
data_train.shape, data_val.shape

In [ ]:
label_train.shape, label_val.shape

$\leadsto$ **Il est important de savoir expliquer les valeurs des dimensions des tableaux...**

## 1.4 $-$ Créer les labels au format 'one hot'

Définir `y_train` et `y_val`, les tableaux des labels d'entraînement et de test au format *one hot* : 

Vérifier visuellement les 5 premières valeurs des tableaux `label_train` et `y_train` puis `label_val` et `y_val`:

## 1.5 $-$ Définir les paramètres utiles

En utilisant les attributs des tableaux *ad-hoc*, définir les paramètres suivants :

In [ ]:
nb_train_set = ...    # nombre de jeux d'entraînement
nb_val_set   = ...    # nombre de jeux de validation
set_size     = ...    # nombre de réels (float) dans un jeu
nb_classe    = ...    # nombre de classes de matériaux 

Vérification :

In [ ]:
print(f"{nb_train_set} jeux d'entraînement  et {nb_val_set} jeux de validation, comprenant {set_size} scalaires dans chaque jeu")
print(f"{nb_train_set} jeux d'entraînement  et {nb_val_set} jeux de validation, comprenant {set_size} scalaires dans chaque jeu")
print(f"{nb_classe} classes de matériaux")

# 2 $-$ Construire et entraîner le réseau de neurones dense

## 2.1 $-$ Construire le du réseau dense

En vous appuyant sur les acquis d'apprentissage de l'auto-formation, contruire le RND `model` conforme aux spécifications :
- couche d'entrée compatible avec les dimensions du jeu de connées construit plus haut, nommée `Input`,
- couche cachée de 100 neurones, fonction d'activation `relu`, nommée `C1`,
- couche de sortie permettant de classifier deux matières percées, nommée `Out`,<br>

Compiler le réseau avec les paramètres *ad-hoc* et faire afficher sa structure avec la méthode `summary`.

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input

# set the seed for repetable tenssorflow random sequences:
tf.random.set_seed(SEED)

model = ...


Affichage graphique de la structure du réseau :

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_activations=True)

## 2.2 $-$ Sauvegarder les poids initiaux du réseau de neurones

- Enregistrer les poids du réseau de neurone initial dans le dossier `weights` avec le préfixe `dense_init`.
- Afficher la liste des fichiers du dossier `weights` qui commencent par `dense_init`.

## 2.3 $-$ Entraîner le réseau de neurones

- Recharger les poids initiaux du réseau.
- Fixer la graine des génératoires aléatoire de **tensorflow**.
- Entraîner le réseau, avec mesure des performance à chaque époque, en essayant de trouver par essais successifs des valeurs des arguments `epoch` et `batch_size` qui donnent des courbes `val_accuracy` et `val_loss` satisfaisantes.
- Afficher les courbes de précisoion et de perte...

### Sauvegarder les poids du réseau entraîné :

- Enregistrer les poids du réseau pour le meilleur entraînement dans le dossier `models` avec le préfixe `trained`.
- Faire afficher la liste des fichiers du dossier `weights` qui commencent par `trained`.

# 3 $-$ Évaluer les performances du réseau entraîné

Calculer les inférences (scalaires) du réseau entraîné en lui donnant en entrée les données de validation :

Calculer la précison (pourcentage de bonnes réponses) du réseau entraîné :

Faire afficher la matrice de confusion, avec les labels écrits explicitement :

# 4 - Entraîner le réseau de neurones avec seulement 1 indicateur à la fois parmi les 50

À cette étape du problème, l'idée est d'entraîner le RND non plus avec la globalité des 50 indicateurs, mais de considérer les indicateurs un par un, pour voir quelle est la précision du réseau entraîné avec les données de chaque indicateur pris séparément...

## 4.1 $-$ Construire le réseau de neurones pour 1 indicateur en entrée :

Définir le RND `model_1` identique au RND `model` sauf pour la couche d'entrée qui sera maintenant dimensionnée à `1`, compiler le réseau et afficher sa structure :

Affichage graphique de la structure du réseau :

In [ ]:
tf.keras.utils.plot_model(model_1, show_shapes=True, show_layer_activations=True)

### Sauvegarder les poids initiaux du réseau de neurones

- Enregistrer les poids du réseau de neurones initial dans le dossier `weights` avec le préfixe `dense_1_init`.
- Afficher la liste des fichiers du dossier `weights` qui commencent par `dense_1_init`.

## 4.2 $-$ Entraîner le réseau avec chacun des 50 indicateurs pris séparément

Construire une boucle réalisant 50 itérations, et à chaque itération `i` :
- Extraire les données d'entraînement et de validation correspondant à l'indicateur de rang `i`,
- Charger les poids initiaux du RND `model_1`,
- Fixer la graine des générateurs aléatoires de **tensorflow**,
- Entraîner le réseau avec les données d'entraînement et de validation de l'indicateur `i`, avec mesure des performance à chaque époque, en choisissant<br>
  les meilleures valeurs de `epoch` et `batch_size` compte tenu des résultats précédents.
- Calculer les inférences scalaires du réseau pour les données de validation, en déduire la précision du réseau entraîné avec l'indicateur de rang `i`,
- Stocker dans la liste `accuracy` la précision du réseau entraîné.
- Afficher la précision du réseau entraîné avec chacun des 50 indicateurs : à l'aide de la page [sphx-glr-plot-types-basic-bar-py](https://matplotlib.org/stable/plot_types/basic/bar.html#sphx-glr-plot-types-basic-bar-py) tracer un "diagramme en bâtons" avec la fonction `bar` du module **matplotlib**, montrant :
    - en abcisses, le rang des indicateurs de 0 à 49,
    - en ordonnées, la précision en % du réseau entrainé avec chacun des 50 indicateurs...<br>
      *indications* : on pourra passer l'argument `figsize=(15,5)` à l'appel de la fonction `plt.subplots` pour régler la taille de la figure...

## 4.3 $-$ Rechercher les indicateurs les plus pertinents

L'allure de la figure précédente suggère que certains indicateurs n'offrent aucune pertinence pour la classification du matériau percé (précision égale à 50 % $\leadsto$ la même que si on classait les matériaux au hasard...), alors que d'autres conduisent à eux seuls à des précisions de classification supérieures à 80 voire 90 %.

Faire afficher les labels des indicateurs donnant un réseau entraîné dont la précision est meilleure que 90% :

Classer par ordre décroissant de précision les indicateurs sélectionnés (afficher la précision, le label et le rang des indicateurs):

# 5 $-$ Entraînement final avec les indicateurs les plus pertinents

Parmis les capteurs utilisables sur le banc de perçage, certains sont compliqués à rajouter (par exemple le capteur de force axiale), d'autres sont relativement simples à ajouter (les accéléromètres par exemple) et d'autres encore sont installés de façon native quand le banc est construit (comme les capteurs de courant...) et sont donc particulièrement intéressants à utiliser..

Refaire l'entraînement du RND en ne conservant que le[s] indicateur[s] que vous jugez pertinent[s], donnant une précision meilleure que 90 %.<br>
Expliquez pourquoi les indicateurs que vous avez retenus sont des indicateurs *pertinents*<br>
Faire afficher la matrice de confusion...

# 6 $-$ Bilan

- Quelles conclusions tirez-vous de la résolution de ce problème ?

 
- Quelles applications pourriez-vous envisager pour ce réseau entraîné à classifier le matériau percé ?

 
- Quelles suite pourriez-vous donner à cette étude ?

 